In [1]:
!git clone https://github.com/peppermenta/faceHack

Cloning into 'faceHack'...
remote: Enumerating objects: 6461, done.
remote: Counting objects: 100% (6461/6461), done.
remote: Compressing objects: 100% (6458/6458), done.
remote: Total 6461 (delta 2), reused 6458 (delta 1), pack-reused 0
Receiving objects: 100% (6461/6461), 61.98 MiB | 25.33 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [4]:
!pip3 install -r faceHack/requirements.txt

In [22]:
import torch
from PIL import Image
import numpy as np
import os      
import torchvision         

class FaceDataset(torch.utils.data.Dataset):
  def __init__(self,train=True,type='normal',transform=None):
    super().__init__()
    if(train):
      self.data = np.genfromtxt(os.path.join('faceHack','dataset','train.csv'),delimiter=',',dtype=None,encoding='utf-8')
    else:
      self.data = np.genfromtxt(os.path.join('faceHack','dataset','test.csv'),delimiter=',',dtype=None,encoding='utf-8')

    self.type=type
    self.transform = transform

  def __len__(self):
    return self.data.shape[0]

  def __getitem__(self, index):
    img_name = self.data[index][0]
    X = Image.open(os.path.join('faceHack','dataset','happy_images','{}.jpg'.format(img_name)))
    label = self.data[index][1]

    if(self.type=='normal'):
      if(label=='NOT smile'):
        y = 0
      else:
        y = 1
    else:
      if(label=='positive smile'):
        y = 1
      else:
        y = 0

    if(self.transform):
      X = self.transform(X)
    return X,y


In [25]:
faceTransform = torchvision.transforms.Compose([
  torchvision.transforms.Resize(224),
  torchvision.transforms.RandomHorizontalFlip(),
  torchvision.transforms.ToTensor(),
])

In [30]:
model = torch.hub.load('pytorch/vision:v0.9.0', 'resnext101_32x8d', pretrained=True)

for param in model.parameters():
  param.requires_grad = False

in_features = model.fc.in_features
model.fc = torch.nn.Linear(in_features=in_features,out_features=2)

trainDataset = FaceDataset(transform=faceTransform)
testDataset = FaceDataset(train=False,transform=faceTransform)
trainLoader = torch.utils.data.DataLoader(trainDataset,batch_size=32,shuffle=True,num_workers=2)
testLoader = torch.utils.data.DataLoader(testDataset,batch_size=32,shuffle=False,num_workers=2)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


In [33]:
model.train()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
epochs = 20

loss_hist = []

for epoch in range(epochs):
  tot_loss = 0
  for i,data in enumerate(trainLoader):
    print(i)
    X,y = data

    out = model(X)

    loss = criterion(out,y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    tot_loss += loss.item()
    if(i%20==0):
      print('Epoch:({},{}): Loss={}'.format(epoch,i,loss.item()))

  
  loss_hist.append(tot_loss)
import matplotlib.pyplot as plt

plt.plot([i for i in range(epochs)],loss_hist)


0


KeyboardInterrupt: ignored

In [ ]:
# Imports
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Dataset Preparation
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

# Define some Neural Network for training
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Training Loop
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        # Forward Propagation
        outputs = net(inputs)

        # Loss
        loss = criterion(outputs, labels)

        # Backward Propagation
        loss.backward()
        
        if i%2000 == 0:
          print(f"[{epoch}, {i}] Loss: {loss.item()}")
        
        optimizer.step()

print('Finished Training')

# Train Accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Train Accuracy: {100*(correct/total)}")

# Test Accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Test Accuracy: {100*(correct/total)}")

torch.save(net, "trained_model.pt")
print("Model saved!")

trained_network = torch.load("trained_model.pt")
print("Model Loaded!")

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
[0, 0] Loss: 2.3519058227539062
[0, 2000] Loss: 1.6639143228530884
[0, 4000] Loss: 1.704770803451538
[0, 6000] Loss: 1.3578636646270752
[0, 8000] Loss: 2.0630784034729004
[0, 10000] Loss: 1.6594014167785645
[0, 12000] Loss: 1.031550645828247
[1, 0] Loss: 0.9881848096847534
[1, 2000] Loss: 0.37073346972465515
[1, 4000] Loss: 1.6300749778747559
[1, 6000] Loss: 1.2850522994995117
[1, 8000] Loss: 0.9940696358680725
[1, 10000] Loss: 0.6007208824157715
[1, 12000] Loss: 0.9381982088088989
Finished Training
Train Accuracy: 56.816
Test Accuracy: 55.16
Model saved!
Model Loaded!


In [ ]:
# Imports
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Dataset Preparation
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

# Define some Neural Network for training
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Training Loop
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        # Forward Propagation
        outputs = net(inputs)

        # Loss
        loss = criterion(outputs, labels)

        # Backward Propagation
        loss.backward()
        
        if i%2000 == 0:
          print(f"[{epoch}, {i}] Loss: {loss.item()}")
        
        optimizer.step()

print('Finished Training')

# Train Accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Train Accuracy: {100*(correct/total)}")

# Test Accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Test Accuracy: {100*(correct/total)}")

torch.save(net, "trained_model.pt")
print("Model saved!")

trained_network = torch.load("trained_model.pt")
print("Model Loaded!")

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
[0, 0] Loss: 2.3519058227539062
[0, 2000] Loss: 1.6639143228530884
[0, 4000] Loss: 1.704770803451538
[0, 6000] Loss: 1.3578636646270752
[0, 8000] Loss: 2.0630784034729004
[0, 10000] Loss: 1.6594014167785645
[0, 12000] Loss: 1.031550645828247
[1, 0] Loss: 0.9881848096847534
[1, 2000] Loss: 0.37073346972465515
[1, 4000] Loss: 1.6300749778747559
[1, 6000] Loss: 1.2850522994995117
[1, 8000] Loss: 0.9940696358680725
[1, 10000] Loss: 0.6007208824157715
[1, 12000] Loss: 0.9381982088088989
Finished Training
Train Accuracy: 56.816
Test Accuracy: 55.16
Model saved!
Model Loaded!


In [ ]:
model.eval()